# Analyzing the Jacobian of the SLP

In this section I evaluate the stability of the single layer perceptron models (SLP) using the jacobian

$$
J(x_0) = \frac{\partial f(\vec{x})}{\partial \vec{x}}\bigg|_{x=x_0}.
$$
The jacobian is basically synonomous with the linear response function.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import numpy as np
import xarray as xr
import torch
from torch.autograd import Variable

from lib.models.torch_models import predict, jacobian

In [ ]:
import holoviews as hv
from lib.hvops import quadmesh
hv.extension('matplotlib')

Let's load the data.

In [ ]:
data = np.load("../data/ml/ngaqua/time_series_data.npz")


X = data['X']
G = data['G']
scale = data['scales']
w = data['w']

# # we need to grap the pressure field from a different path
p = xr.open_dataset("../data/raw/ngaqua/stat.nc").p.values
# t = dt * np.arange(X.shape[0])

and compute the mean.

In [ ]:
x_mean = np.apply_over_axes(np.mean, X, axes=(0,1,2))

# Analyzing the Jacobian

In [ ]:
net = torch.load("../data/ml/ngaqua/time_series_fit.torch")

In [ ]:


jac = jacobian(net, x_mean)  

Let's plot this jacobian

In [ ]:
opts = dict(plot=dict(invert_yaxis=True, invert_xaxis=True, colorbar=True),
            style=dict(cmap='viridis'))


quadmesh(p, p, jac[:34,:34], kdims=['p (s_l)', r'p ($\frac{d(s_l)}{dt}$)']).opts(**opts)

this pane looks very similar to the results we had with the linear response function.

In [ ]:
quadmesh(p, p, jac[:34,34:], kdims=['p (q_t)', 'p (d(s_l)/dt)']).opts(**opts)

It looks like the main problem is the strong dependence on the humidity in region where humidity is basically 0! I could presumably fix these issues by only considering the lower atmospheric humidity.

What are the eigenvalues of this jacobian?

In [ ]:
from scipy.linalg import eigvals

In [ ]:
lam = eigvals(jac)
lam[np.argsort(lam.real)]

These are obviously huge, but again fixing the humidity may help.